<a href="https://colab.research.google.com/github/Jacofeldman/Metodos1_JacoboFeldman/blob/main/Tarea6/Hidden_Markov_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np

T = np.array([[0.8, 0.2],  # J -> [J, B]
              [0.2, 0.8]]) # B -> [J, B]

E = np.array([[0.5, 0.5],  # J -> [C, S]
              [0.9, 0.1]]) # B -> [C, S]

pi = np.array([0.2, 0.8])  # [J, B]

omega_o = ['S', 'C', 'C', 'C', 'S', 'C', 'S', 'C']
obs_map = {'C': 0, 'S': 1}
observed_seq = [obs_map[o] for o in omega_o]


def forward_algorithm(T, E, pi, observed_seq):
    N = T.shape[0]
    T_len = len(observed_seq)

    alpha = np.zeros((T_len, N))
    scale_factors = np.zeros(T_len)


    alpha[0, :] = pi * E[:, observed_seq[0]]
    scale_factors[0] = np.sum(alpha[0, :])
    alpha[0, :] /= scale_factors[0]  # Normalización


    for t in range(1, T_len):
        for j in range(N):
            alpha[t, j] = E[j, observed_seq[t]] * np.sum(alpha[t - 1, :] * T[:, j])
        scale_factors[t] = np.sum(alpha[t, :])
        alpha[t, :] /= scale_factors[t]  # Normalización


    prob_o = np.prod(scale_factors)
    return alpha, prob_o, scale_factors


def backward_algorithm(T, E, observed_seq, scale_factors):
    N = T.shape[0]
    T_len = len(observed_seq)

    beta = np.zeros((T_len, N))
    beta[-1, :] = 1.0 / scale_factors[-1]  # Normalización del último paso


    for t in range(T_len - 2, -1, -1):
        for i in range(N):
            beta[t, i] = np.sum(T[i, :] * E[:, observed_seq[t + 1]] * beta[t + 1, :])
        beta[t, :] /= scale_factors[t]  # Normalización

    return beta


def viterbi_algorithm(T, E, pi, observed_seq):
    N = T.shape[0]
    T_len = len(observed_seq)

    delta = np.zeros((T_len, N))
    psi = np.zeros((T_len, N), dtype=int)
    delta[0, :] = pi * E[:, observed_seq[0]]


    for t in range(1, T_len):
        for j in range(N):
            delta[t, j] = E[j, observed_seq[t]] * np.max(delta[t - 1, :] * T[:, j])
            psi[t, j] = np.argmax(delta[t - 1, :] * T[:, j])


    states = np.zeros(T_len, dtype=int)
    states[-1] = np.argmax(delta[-1, :])
    for t in range(T_len - 2, -1, -1):
        states[t] = psi[t + 1, states[t + 1]]

    seq_prob = np.max(delta[-1, :])
    return states, seq_prob


alpha, prob_o, scale_factors = forward_algorithm(T, E, pi, observed_seq)
beta = backward_algorithm(T, E, observed_seq, scale_factors)
hidden_states, seq_prob = viterbi_algorithm(T, E, pi, observed_seq)


P_o = (alpha * beta) / np.sum(alpha[-1, :])  # Probabilidad normalizada


state_map = {0: 'J', 1: 'B'}
hidden_states_mapped = [state_map[s] for s in hidden_states]


suma_prob = np.sum(P_o, axis=1)


print("a) Probabilidad total de la secuencia observada:", prob_o)
print("b) Secuencia oculta más probable:", hidden_states_mapped)
print("   Probabilidad de la secuencia más probable:", seq_prob)
print("c) Probabilidades de los estados observables:\n", P_o)
print("d) Suma de probabilidades por tiempo (verificación):", suma_prob)
if np.allclose(suma_prob, 1.0):
    print("   La suma de probabilidades es 1 en todos los pasos.")
else:
    print("   Error: La suma de probabilidades no es 1 en algunos pasos.")
print("e) ¿El resultado depende de la probabilidad a-priori?: Sí")


a) Probabilidad total de la secuencia observada: 0.0019344381513984007
b) Secuencia oculta más probable: ['B', 'B', 'B', 'B', 'J', 'J', 'J', 'J']
   Probabilidad de la secuencia más probable: 0.00019110297600000007
c) Probabilidades de los estados observables:
 [[2.41828548 3.13727008]
 [0.48246048 0.9738502 ]
 [0.46053573 0.91536595]
 [0.5982346  0.73775906]
 [3.08876906 1.06546453]
 [1.02922181 0.52206286]
 [2.60194107 0.74425434]
 [0.90812442 0.70377562]]
d) Suma de probabilidades por tiempo (verificación): [5.55555556 1.45631068 1.37590168 1.33599366 4.15423359 1.55128468
 3.34619541 1.61190005]
   Error: La suma de probabilidades no es 1 en algunos pasos.
e) ¿El resultado depende de la probabilidad a-priori?: Sí
